<a href="https://colab.research.google.com/github/rajni0829/Python/blob/main/UnbeatableAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import sys
import pygame
import random
import numpy as np

WIDTH    = 800
HEIGHT   = 800
ROWS     = 4
COLS     = 4
OFFSET   = 50
SQSIZE   = WIDTH // COLS
RADIUS   = SQSIZE // 3

LINE_WIDTH  = 15
CIRC_WIDTH  = 15
CROSS_WIDTH = 25



# --- COLOR CODES ---

BG_COLOR    = (0,0,0)
LINE_COLOR  = (255,255,255)
CIRC_COLOR  = (255,0,0)
CROSS_COLOR = (0, 181, 184)


# --- PYGAME SETUP ---

pygame.init()
screen = pygame.display.set_mode( (WIDTH, HEIGHT) )
pygame.display.set_caption('''                                                                                       TIC TAC TOE - AN UNBEATABLE AI''')
screen.fill( BG_COLOR )

# --- CLASSES ---

class Board:

    def __init__(self):
        self.squares = np.zeros( (ROWS, COLS) ) 
        # print(self.squares)
        self.empty_sqrs = self.squares                       # [squares]
        self.marked_sqrs = 0

    def final_state(self, show=False):
        '''
            no win yet ->    return 0 
            P1 wins    ->    return 1
            P2 wins    ->    return 2 
        '''

        # vertical wins
        for col in range(COLS):
            if self.squares[0][col] == self.squares[1][col] == self.squares[2][col] == self.squares[3][col] != 0:
                if show:
                    color = CIRC_COLOR if self.squares[0][col] == 2 else CROSS_COLOR
                    iPos = (col * SQSIZE + SQSIZE // 2, 20)
                    fPos = (col * SQSIZE + SQSIZE // 2, HEIGHT - 20)
                    pygame.draw.line(screen, color, iPos, fPos, LINE_WIDTH)
                return self.squares[3][col]

        # horizontal wins
        for row in range(ROWS):
            if self.squares[row][0] == self.squares[row][1] == self.squares[row][2] == self.squares[row][3] != 0:
                if show:
                    color = CIRC_COLOR if self.squares[row][0] == 2 else CROSS_COLOR
                    iPos = (20, row * SQSIZE + SQSIZE // 2)
                    fPos = (WIDTH - 20, row * SQSIZE + SQSIZE // 2)
                    pygame.draw.line(screen, color, iPos, fPos, LINE_WIDTH)
                return self.squares[row][0]

        # desc diagonal
        if self.squares[0][0] == self.squares[1][1] == self.squares[2][2] == self.squares[3][3] != 0:
            if show:
                color = CIRC_COLOR if self.squares[1][1] == 2 else CROSS_COLOR
                iPos = (20, 20)
                fPos = (WIDTH - 20, HEIGHT - 20)
                pygame.draw.line(screen, color, iPos, fPos, CROSS_WIDTH)
            return self.squares[1][1]

        # asc diagonal
        if self.squares[2][1] == self.squares[1][2] == self.squares[0][3] == self.squares[3][0] != 0:
            if show:
                color = CIRC_COLOR if self.squares[1][1] == 2 else CROSS_COLOR
                iPos = (20, HEIGHT - 20)
                fPos = (WIDTH - 20, 20)
                pygame.draw.line(screen, color, iPos, fPos, CROSS_WIDTH)
            return self.squares[1][1]

        # no win yet
        return 0

    def mark_sqr(self, row, col, player):      # assigning 0(ie. arr's val to players)
        self.squares[row][col] = player        
        self.marked_sqrs += 1

    def empty_sqr(self, row, col):
        return self.squares[row][col] == 0

    def get_empty_sqrs(self):   
        empty_sqrs = []
        for row in range(ROWS):
            for col in range(COLS):
                if self.empty_sqr(row, col):
                    empty_sqrs.append( (row, col) )
        
        return empty_sqrs

    def isfull(self):
        return self.marked_sqrs == 9

    def isempty(self):
        return self.marked_sqrs == 0

class AI:

    def __init__(self, level=1, player=2):
        self.level = level
        self.player = player

    # --- RANDOM ---

    def rnd(self, board):     # for level 0
        empty_sqrs = board.get_empty_sqrs()
        idx = random.randrange(0, len(empty_sqrs)) 

        return empty_sqrs[idx] # (row, col)


    # --- MINIMAX ---

    def minimax(self, board, maximizing):
        
        # terminal case
        case = board.final_state()
        # print(case)

        # player 1 wins  ->  minimising player
        if case == 1:
            return 1, None # eval, move

        # player 2 wins  ->  maximising player
        if case == 2:
            return -1, None

        # if case == 0:
        #     game.next_turn();

        # draw
        elif board.isfull():
            return 0, None

        # print(maximizing)
        if maximizing:
            max_eval = -100
            best_move = None
            empty_sqrs = board.get_empty_sqrs()
            # print(empty_sqrs)

            for (row, col) in empty_sqrs:
                temp_board = copy.deepcopy(board)
                temp_board.mark_sqr(row, col, 1) # x, y, player
                eval = self.minimax(temp_board, False)[0]
                if eval > max_eval:
                    max_eval = eval
                    best_move = (row, col)
                    
            # print(max_eval, best_move)

            return max_eval, best_move

        elif not maximizing:
            min_eval = 100     # anything > 1,-1,0
            best_move = None
            empty_sqrs = board.get_empty_sqrs()

            for (row, col) in empty_sqrs:                                        # (row,col) -> best move
                temp_board = copy.deepcopy(board)                                # copying but don't want main board to be affected
                temp_board.mark_sqr(row, col, self.player)  # x, y, player
                # temp_board.mark_sqr(row, col, self.player)                         # x, y, player
                eval = self.minimax(temp_board, True)[0]                            # returns eval
                                                                              #since minimax is a tuple - returns (eval,best move)
                if eval < min_eval:
                    min_eval = eval
                    best_move = (row, col)
 
            return min_eval, best_move

    # --- MAIN EVAL ---

    def eval(self, main_board):
        if self.level == 0:
            # random choice
            eval = 'random'
            move = self.rnd(main_board)
        else:
            # pass
            # minimax algo choice
            eval, move = self.minimax(main_board, False)

        print(f'Utility of the move chosen by AI : {move} having eval: {eval}')

        return move # row, col

class Game:

    def __init__(self):
        self.board = Board()
        self.ai = AI()
        self.player = 1                                                              # 1-cross  #2-circles   # next player to mark
        self.gamemode = 'ai'                                                         # pvp or ai
        self.running = True                                                          # when game isn't over -> player wins or draw(board is full)
        self.show_lines()


    # --- DRAW METHODS ---

    def show_lines(self):
        # bg
        screen.fill( BG_COLOR )

        # vertical
        pygame.draw.line(screen, LINE_COLOR, (SQSIZE, 0), (SQSIZE, HEIGHT), LINE_WIDTH)            
        pygame.draw.line(screen, LINE_COLOR, (2*SQSIZE, 0), (2*SQSIZE, HEIGHT), LINE_WIDTH)
        pygame.draw.line(screen, LINE_COLOR, (3*SQSIZE , 0), (3*SQSIZE , HEIGHT), LINE_WIDTH)

        # horizontal
        pygame.draw.line(screen, LINE_COLOR, (0, SQSIZE), (WIDTH, SQSIZE), LINE_WIDTH)
        pygame.draw.line(screen, LINE_COLOR, (0, 2*SQSIZE), (WIDTH, 2*SQSIZE), LINE_WIDTH)
        pygame.draw.line(screen, LINE_COLOR, (0, 3*SQSIZE), (WIDTH, 3*SQSIZE), LINE_WIDTH)

    def draw_fig(self, row, col):
        if self.player == 1:
            # pass
            # draw cross
            # desc line
            start_desc = (col * SQSIZE + OFFSET, row * SQSIZE + OFFSET)
            end_desc = (col * SQSIZE + SQSIZE - OFFSET, row * SQSIZE + SQSIZE - OFFSET)
            pygame.draw.line(screen, CROSS_COLOR, start_desc, end_desc, CROSS_WIDTH) 
            # asc line
            start_asc = (col * SQSIZE + OFFSET, row * SQSIZE + SQSIZE - OFFSET)
            end_asc = (col * SQSIZE + SQSIZE - OFFSET, row * SQSIZE + OFFSET)
            pygame.draw.line(screen, CROSS_COLOR, start_asc, end_asc, CROSS_WIDTH)
        
        elif self.player == 2:
            # draw circle
            center = (col * SQSIZE + SQSIZE // 2, row * SQSIZE + SQSIZE // 2)
            pygame.draw.circle(screen, CIRC_COLOR, center, RADIUS, CIRC_WIDTH)

    # --- OTHER METHODS ---

    def make_move(self, row, col):
        self.board.mark_sqr(row, col, self.player)   # x,y,player
        self.draw_fig(row, col)
        self.next_turn()

    def next_turn(self):
        self.player = self.player % 2 + 1   # remainder + 1
        print(self.player)

    def change_gamemode(self):
        self.gamemode = 'ai' if self.gamemode == 'pvp' else 'pvp'

    def isover(self):
        return self.board.final_state(show=True) != 0 or self.board.isfull()

    def reset(self):
        self.__init__()

def main():

    print('''
    Press g -> To change the gamemode
    Press r -> To restart the game
    Press 0 -> To play L-0 Game
    Press 1 -> To play L-1 Game
    ''')

    # --- OBJECTS ---

    game = Game() 
    board = game.board
    ai = game.ai

    # --- MAINLOOP ---

    while True:
        
        # pygame events
        for event in pygame.event.get():

            # quit event
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

            # keydown event
            if event.type == pygame.KEYDOWN:

                # g-gamemode
                if event.key == pygame.K_g:
                    game.change_gamemode()

                # r-restart
                if event.key == pygame.K_r:
                    game.reset()
                    board = game.board
                    ai = game.ai

                # 0-random ai
                if event.key == pygame.K_0:
                    ai.level = 0
                
                # 1-random ai
                if event.key == pygame.K_1:
                    ai.level = 1

            # click event
            if event.type == pygame.MOUSEBUTTONDOWN:
                pos = event.pos
                row = pos[1] // SQSIZE                                        # y / 200
                col = pos[0] // SQSIZE                                        # x / 200
                # print(row,col)
                
                # human mark sqr
                if board.empty_sqr(row, col) and game.running:
                    game.make_move(row, col)
                    # board.mark_sqr(row,col,game.player)
                    # game.draw_fig(row,col)
                    # game.next_turn()
                    # print(board.squares)

                    if game.isover():
                        game.running = False

        # print(game.gamemode,game.player,game.running,ai.player)

        # AI initial call
        if game.gamemode == 'ai' and game.player == ai.player and game.running:


            # update the screen - before it plays next move
            pygame.display.update()

            row, col = ai.eval(board) 
            print(row,col)  
                                                           # eval
            print(board.squares)
            game.make_move(row, col)
            # board.mark_sqr(row,col,ai.player)
            # game.draw_fig(row,col)
            # break
            # game.next_turn()
            # print(board.squares)

            if game.isover():
                game.running = False
            
        pygame.display.update()

main()